<a href="https://colab.research.google.com/github/WebiksInc/Stop-Words-Hebrew/blob/main/prepare_stop_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# List Stop Words in Hebrew

In [11]:
# Install pyconll to handle UD
try:
    import pyconll
except ImportError as e:
    !pip install pyconll

In [12]:
# Import package
import numpy as np
import pandas as pd
import requests as req
import pyconll

In [13]:
# URL data train/test/dev
ud_dev = "https://raw.githubusercontent.com/UniversalDependencies/UD_Hebrew-IAHLTwiki/master/he_iahltwiki-ud-dev.conllu"
ud_test = "https://github.com/UniversalDependencies/UD_Hebrew-IAHLTwiki/raw/master/he_iahltwiki-ud-test.conllu"
ud_train = "https://github.com/UniversalDependencies/UD_Hebrew-IAHLTwiki/blob/master/he_iahltwiki-ud-train.conllu?raw=true"
top_freq_word_wiki = "https://raw.githubusercontent.com/NNLP-IL/Stop-Words-Hebrew/main/top_3000_most_freq_wiki.csv"

In [14]:
# download data
def get_data(url: str, name: str):
    res = req.get(url)
    file = open(name + ".txt", "w")
    file.write(res.text)
    file.close()


get_data(ud_dev, "ud_dev")
get_data(ud_test, "ud_test")
get_data(ud_train, "ud_train")

In [15]:
# load the UD data
dev = pyconll.load_from_file("/content/ud_dev.txt")
train = pyconll.load_from_file("/content/ud_test.txt")
test = pyconll.load_from_file("/content/ud_train.txt")

In [16]:
# Word extraction from the UD. Cleaning and adding words
def remove_niqqud_from_string(my_string):
    return "".join(["" if 1456 <= ord(c) <= 1479 else c for c in my_string])


def Extracts_stopwords(dat: pyconll):
    # Extracts the stopwords form the UD Schema
    POS_tag = ["DET", "ADP", "PRON", "CCONJ", "SCONJ", "SYM"]
    lst_words = [
        token.form for sentence in dat for token in sentence if token.upos in POS_tag
    ]
    return lst_words


words_plus = {
    "אגב",
    "הואיל",
    "הלא",
    "הרי",
    "כיאלו",
    "כיון",
    "יען",
    "על" "מנת",
    "ודאי",
    "גם",
    "אפילו",
    "יש",
    "רק",
}
words_minus = {
    "%",
    "&",
    "(",
    "+",
    "-",
    "/",
    ":",
    "=",
    "–",
    "°",
    "בינ",
    "שורת",
    "אודות",
    'אודותי',
    'דם',
    "לאזור",
    "במקביל",
    "כשב-1948",
    "ד",
    "בתוכ",
    "הו",
    "י",
    "ך",
    "ם",
    "ן",
    "נ'",
    "נאומ",
    'למחרת',
    'אות',
    'אית',
    'ספר',
    'ערב'
}

# Extracts the stopwords form the UD Schema
stopwords_raw = set(
    Extracts_stopwords(dev) + Extracts_stopwords(train) + Extracts_stopwords(test)
)
# Optionally, delete the words with the niqqud
stopwords_raw = set(map(remove_niqqud_from_string,stopwords_raw))
# Delete incorrect words
stopwords_base = stopwords_raw - words_minus
# Add missing words that we identified . Note that there is another addition of words below
stopwords_base = stopwords_base | words_plus

# Note: The list includes English words that happened to be in the database. For a complete list in English we recommend nltk package.

In [17]:
# most freq words form the hebrew wikipedia
wiki_freq = pd.read_csv(top_freq_word_wiki)
wiki_top_50 = wiki_freq.head(50)['word'].to_list()
wiki_remove = {'בית','במהלך','בשנת','הברית','העולם','הראשון','ישראל','מספר'}
wiki_top_50 = set(wiki_top_50) - wiki_remove
stopwords_extend = stopwords_base # | wiki_top_50

In [18]:
# Add prepositions. A detailed explanation of the need appears in README

al = [
    "אלי",
    "אליך",
    "אליך",
    "אליו",
    "אליה",
    "אלינו",
    "אליכם",
    "אליכן",
    "אליהם",
    "אליהן",
]
hal = [
    "עלי",
    "עליך",
    "עליך",
    "עליו",
    "עליה",
    "עלינו",
    "עליכם",
    "עליכן",
    "עליהם",
    "עליהן",
]
him = [
    "עמי",
    "עמך",
    "עמו",
    "עמה",
    "עמנו",
    "עמכם",
    "עמכן",
    "עמם",
    "עמן",
    "עמהם",
    "עמהן"
]
ath_with = [
    "אתן",
    "אתם",
    "אתכן",
    "אתכם",
    "אתנו",
    "אתה",
    "אתו",
    "אתך",
    "אתי"
]
ath = [
    "אותי",
    "אותך",
    "אותו",
    "אותה",
    "אותנו",
    "אתכם",
    "אתכן",
    "אותם",
    "אותן"
]
shel = [
    "שלי",
    "שלך",
    "שלו",
    "שלה",
    "שלנו",
    "שלכם",
    "שלכן",
    "שלהם",
    "שלהן",
]
men = [
    "ממני",
    "ממך",
    "ממנו",
    "ממנה",
    "מכם",
    "מכן",
    "מהם",
    "מהן"
]
azhel = [
    "אצלי",
    "אצלך",
    "אצלו",
    "אצלה",
    "אצלנו",
    "אצלכם",
    "אצלכן",
    "אצלם", 
    "אצלן"
]
bshvil = [
    "בשבילי",
    "בשבילך",
    "בשבילו",
    "בשבילה",
    "בשבילנו",
    "בשבילכם",
    "בשבילכן",
    "בשבילם",
    "בשבילן",
]
bin = [
    "ביני",
    "בינך",
    "בינו",
    "בינה",
    "בינינו",
    "ביניכם",
    "ביניכן",
    "ביניהם",
    "ביניהן",
    "בינם",
    "בינן",
]
lafi = [
    "לפי",
    "לפיך",
    "לפיו",
    "לפיה",
    "לפינו",
    "לפיכם",
    "לפיכן",
    "לפיהם",
    "לפיהן"
]
lafni = [
    "לפני",
    "לפניך",
    "לפניו",
    "לפניה",
    "לפנינו",
    "לפניכם",
    "לפניכן",
    "לפניהם",
    "לפניהן",
]
chmo = [
    "כמוני",
    "כמוך",
    "כמוהו",
    "כמוה",
    "כמונו",
    "כמוכם",
    "כמוכן",
    "כמוהם",
    "כמוהן"
]
lakrt = [
    "לקראתי",
    "לקראתך",
    "לקראתו",
    "לקראתה",
    "לקראתנו",
    "לקראתכם",
    "לקראתכן",
    "לקראתם",
    "לקראתן",
]
pepositions = {
    "אל": al,
    "על": hal,
    "עם": him,
    "(=עם) את": him,
    "את (תוויות מושא ישיר": ath,
    "של": shel,
    "מן": men,
    "אצל": azhel,
    "בשביל": bshvil,
    "בין": bin,
    "לפי": lafi,
    "לפני": lafni,
    "כמו": chmo,
    "לקראת": lakrt,
}
pepositions_lst = [item for sublist in list(pepositions.values()) for item in sublist]

stopwords_extend = stopwords_extend | set(pepositions_lst)
print('Long list:')
print(
    ", ".join(
        [
            word if index % 15 != 0 else "\n," + word
            for index, word in enumerate(stopwords_extend)
        ]
    )
)

Long list:

,זולת, מש, למרות, עמנו, אליכם, עקב, בשבילנו, עליה, לקראתה, כדי, כי, לנוכח, רוב, תחתי, בהתחשב, 
,עמהם, גבי, לפיהם, עליכם, בטרם, עצמה, בינינו, עמו, בשבילכן, מה, בינם, לפיכם, כלל, מהן, רק, 
,אתכן, כמו, כאילו, to, שלכם, לפניכן, פחות, זו, אולם, מפני, and, על-פי, לכבוד, בגדר, בלעדי, 
,שלך, ידי, עימ, כול, בינן, מפאת, לכדי, עמ, עליכן, מספיק, משום, מתוכ, אשר, כמה, ביניכם, 
,דנן, ש, עמה, לא, נו, את, כמות, כש, ל, למן, מאחורי, רובן, באשר, סביב, אלו, 
,בקרבת, בשבילן, עצמם, כמוה, לרבות, מתוך, מאשר, בעבור, כיון, בזכות, מחמת, דוגמת, הוא, שלו, בפני, 
,עד, כך, אלמלא, אני, ממנה, מ, לפניו, כלשהי, בגין, יתר, לגבי, לידי, The, שלה, לפניהם, 
,בה, נכון, עמהן, לפני, Your, כיאלו, לבין, לפיה, לרגל, מתחת, למען, עובר, אצלן, מול, ב, 
,כמוהו, ללא, ברם, כל, כלשהו, כמוהם, עבר, אצל, בשבילך, האם, ביניהן, זהו, כמוהן, אותם, מידי, 
,לקראתכן, שאר, יחד, ביניהם, במו, of, בינך, אבל, החל, מאחר, מכן, אותי, באם, יש, עלמנת, 
,הללו, עמכם, לעניין, מי, מאת, מהו, יו, אי, פי, מעלי, לקראתכם, מבלי, מלבד, אליך, שלהם, 
,לפניכם, 

In [19]:
# short stop words list. Intersection UD and 1000 most freq words

wiki_top_1000 = wiki_freq.head(1000)['word'].to_list()
stopwords_short = stopwords_base.intersection(set(wiki_top_1000))
stopwords_short_minus = {''}
stopwords_short = stopwords_short - stopwords_short_minus
print('Short list:')
print(
    ", ".join(
        [
            word if index % 15 != 0 else "\n," + word
            for index, word in enumerate(stopwords_short)
        ]
    )
)

Short list:

,אחרי, למרות, עקב, אחר, כי, כדי, רוב, לקראת, אל, על, אם, לאור, עת, עצמה, בתוך, 
,זה, מה, תחת, כלל, במקום, רק, אותו, כמו, זו, פחות, אולם, מפני, כאשר, אלה, ככל, 
,לפי, גם, ידי, מרבית, כנגד, בתור, מנת, משום, the, אשר, כמה, מבין, למשל, לאורך, היא, 
,עוד, עצמו, כן, לא, הם, מן, את, ל, לעומת, דרך, מעט, קרוב, לו, לאחר, סביב, 
,אלו, עצמם, מתוך, מאשר, בזכות, הוא, בניגוד, בפני, עד, כך, אני, כפי, הרבה, מ, א, 
,במשך, לגבי, בעקבות, לידי, The, בה, נכון, זאת, אלא, מהם, לפני, לבין, מתחת, למען, מעבר, 
,מול, ב, ללא, כל, אף, עבר, אצל, קודם, האם, זהו, אותם, שאר, הן, עבור, יחד, 
,of, אבל, החל, מאחר, מכיוון, יש, הללו, מי, מאת, אי, פי, מצד, לשם, ה, בשל, 
,כגון, כלומר, באמצעות, של, מאז, בגלל, בין, לצד, לה, בקרב, יותר, כ, תוך, בידי, ליד, 
,שכן, נגד, מספר, אך, עם, כיוון, ו, אותה, הרי, מטעם, מעל, או, מדי, בעוד, כלפי


In [20]:
# Download the list
from google.colab import files

pd.DataFrame({"stopswords": list(stopwords_extend)}).to_csv(
    "stopswords_list_extend.txt", index=False, sep="\t"
)

pd.DataFrame({"stopswords": list(stopwords_short)}).to_csv(
    "stopswords_list_short.txt", index=False, sep="\t"
)

files.download("stopswords_list_extend.txt")
files.download("stopswords_list_short.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>